# Dallas Restaurant Analysis

### Researcher(s):  

### Date:  February 1, 2020

### Source(s):
- [Dallas Open Data](https://www.dallasopendata.com)
- [Google](https://www.googlepalcesapi.com)
- [Yelp](https://www.yelpapi.com)

## Summary
#### The purpose of this study...
#### Analysis details (python code) is intentionally viewable.
#### Analysis will provide:
* TBD
* TBD
* TBD

## Analysis Results Overview
#### Initial Analysis ...:
  - 
  - 
  - 

#### Differences ....

*See analysis for details.*

### Possible next steps...
#### ....

## Import Dependencies

In [1]:
# Consider grouping dependencies by type
# with a brief explanation regarding use.

# Dependencies
import pandas as pd
import numpy as np
import requests
import json
import csv
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import plotly.express as px
import datetime
import gmaps

# Google API Key, Yelp API Key
from config import gkey, ykey

# Configure gmaps with API key
gmaps.configure(api_key=gkey)

%matplotlib inline

C:\Users\KDSharp\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


ModuleNotFoundError: No module named 'config'

## Dallas Open Data

In [ ]:
# Load in raw Dallas Open Data
raw_dallas_open_data = 'resources/Restaurant_and_Food_Establishment_Inspections__October_2016_to_Present_.csv'

# Read and display the CSV with Pandas
raw_dallas_open_data_pd = pd.read_csv(raw_dallas_open_data, low_memory=False)
#dallas_open_data_pd.head()

In [ ]:
# Clean and extract specific columns for 

restaurant_name = raw_dallas_open_data_pd['Restaurant Name'].str.extract('([a-zA-Z0-9&+."\'""\-" ]+)', expand=False).str.strip()
inspection_type = raw_dallas_open_data_pd['Inspection Type']
inspection_date = raw_dallas_open_data_pd['Inspection Date']#.astype('datetime64[ns]')
inspection_score = raw_dallas_open_data_pd['Inspection Score']
street_number = raw_dallas_open_data_pd['Street Number'].astype('str')
street_name = raw_dallas_open_data_pd['Street Name']
street_unit = raw_dallas_open_data_pd['Street Unit'].fillna('').astype('str')

# Return first 5 integers for zip  .str.replace('f', repr)
#zip_code = raw_dallas_open_data_pd['Zip Code'].str.replace('-', '1').str.slice(stop=5).astype(int)
zip_code = raw_dallas_open_data_pd['Zip Code'].str.extract('([0-9]+)', expand=False).str.strip().str.slice(stop=5).astype(int)
street_address = raw_dallas_open_data_pd['Street Address']

# Use the inspection date's month and year for inspection_month for consistency
inspection_month = raw_dallas_open_data_pd['Inspection Date'].astype('datetime64[ns]').dt.strftime('%m/1/%y')

# Dallas Open Data Given Fiscal Year Report Reported
inspection_year = raw_dallas_open_data_pd['Inspection Year']

# Check record count.  Should be 42,070
len(restaurant_name)

In [ ]:
#Save data tp dataframe
clean_dallas_open_data_pd = pd.DataFrame({
    'restaurant_name':restaurant_name,
    'inspection_type':inspection_type,
    'inspection_date':inspection_date,
    'inspection_score':inspection_score,
    'street_number':street_number,
    'street_name':street_name,
    'street_unit':street_unit,
    'zip_code':zip_code,
    'street_address':street_address,
    'inspection_month':inspection_month,
    'inspection_year':inspection_year
})
# Confirm expected output & record count of 42,070
#clean_dallas_open_data_pd.info
len(clean_dallas_open_data_pd)

In [ ]:
# Keep unique record by street address, Reformat column name for display, & save to dataframe
unique_dallas_open_data_pd = pd.DataFrame({
    'Restaurant Name':restaurant_name,
    'Inspection Type':inspection_type,
    'Inspection Date':inspection_date,
    'Inspection Score':inspection_score,
    'Street Number':street_number,
    'Street Name':street_name,
    'Street Unit':street_unit,
    'Zip':zip_code,
    'Street Address':street_address,
    'Inspection Month':inspection_month,
    'Inspection Year':inspection_year
}).drop_duplicates(subset=['Street Address'])#.reset_index(drop=True, inplace=True)

# Display dataframe (OPTIONAL)
#unique_dallas_open_data_pd

In [ ]:
# Identify businesses not of interest
# Consider code refactor
unwanted_restaurants = unique_dallas_open_data_pd[
    (unique_dallas_open_data_pd['Restaurant Name'] == '7-11') |
    (unique_dallas_open_data_pd['Restaurant Name'] == '7 - ELEVEN') |
    (unique_dallas_open_data_pd['Restaurant Name'] == '7- ELEVEN') |
    (unique_dallas_open_data_pd['Restaurant Name'] == '7- ELEVEN CONVENIENCE STORE') |
    (unique_dallas_open_data_pd['Restaurant Name'] == '7-11 CONVENIENCE STORE') |
    (unique_dallas_open_data_pd['Restaurant Name'] == '7-11CONVENIENCE STORE') |
    (unique_dallas_open_data_pd['Restaurant Name'] == '7-ELEVEN') |
    (unique_dallas_open_data_pd['Restaurant Name'] == '7-ELEVEN CONVENIENCE') |
    (unique_dallas_open_data_pd['Restaurant Name'] == '7-ELEVEN CONVENIENCE STORE') |
    (unique_dallas_open_data_pd['Restaurant Name'] == '7-ELEVEN CONVENIENE STORE') |
    (unique_dallas_open_data_pd['Restaurant Name'] == '7-ELEVEN STORE') |
    (unique_dallas_open_data_pd['Restaurant Name'] == '7 ELEVEN') |
    (unique_dallas_open_data_pd['Restaurant Name'] == '7-7-ELEVEN') |
    (unique_dallas_open_data_pd['Restaurant Name'] == '99 CENTS ONLY') |
    (unique_dallas_open_data_pd['Restaurant Name'] == '99 CENTS ONLY STORE')
].index

# Drop unwanted businesses to reduce api calls to google/yelp
unique_dallas_open_data_pd.drop(unwanted_restaurants, inplace=True)

# Reset datafram index
unique_dallas_open_data_pd.reset_index(drop=True, inplace=True)

In [ ]:
# Output File (CSV) for validation (OPTIONAL)
#unique_dallas_open_data_output = "./output_data/unique_bus_dal_open_data_2019.csv"

# write dataframe to the csv
#unique_dallas_open_data_pd.to_csv(unique_dallas_open_data_output, sep=',', encoding='utf-8', index=False)

## Google Data

In [ ]:
g_lookup_pd_rn = unique_dallas_open_data_pd
g_lookup_pd_rn["Lat"] = ""
g_lookup_pd_rn["Long"] = ""
g_lookup_pd_rn["Type"] = ""
g_lookup_pd_rn["Rating"] = ""
g_lookup_pd_rn["Rating Count"] = ""

g_lookup_pd_rn

In [ ]:
# Create a copy of re-organized Data
g_lookup_pd_filter = g_lookup_pd_rn.copy()

# Filter by Inspection Score
#g_lookup_pd_filter = g_lookup_pd_filter[g_lookup_pd_filter['Inspection Score'] == 83]

# Filter by Zip
g_lookup_pd_filter = g_lookup_pd_filter[g_lookup_pd_filter['Zip'] == 75219]

g_lookup_pd_filter = g_lookup_pd_filter.reset_index()

# Returning top 50 for demonstration
#g_lookup_pd_filter = g_lookup_pd_filter.head(50)

g_lookup_pd_filter

In [ ]:
# create a params dict that will be updated with new city each iteration
params = {"key": gkey}

# Loop through the g_lookup_pd_org and run a lat/long search for each city
for index, row in g_lookup_pd_filter.iterrows():
    try:
        base_url = "https://maps.googleapis.com/maps/api/geocode/json"

        street = row['Street Address']
        zip_code = row['Zip']
        restaurantname = row['Restaurant Name']
        restaurantname = restaurantname.title()

        # update address key value
        params['address'] = f"{street},{zip_code}"

        # make request
        cities_lat_lng = requests.get(base_url, params=params)

        # print the cities_lat_lng url, avoid doing for public github repos in order to avoid exposing key
        
        # convert to json
        cities_lat_lng = cities_lat_lng.json()
        
        g_lookup_pd_filter.loc[index, "Lat"] = cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
        #lat = g_lookup_pd_filter.loc[index, "Lat"]
        
        g_lookup_pd_filter.loc[index, "Long"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]
        #long = g_lookup_pd_filter.loc[index, "Long"]
        
        print(f'''[OK!] {restaurantname} found!''')            

    except:
        print(f'''[ERROR!] {restaurantname} missing field/result.''')

        
        pass

In [ ]:
# params dictionary to update each iteration
# radius of 50 has been giving me a 90% match for Type, Rating, and Rating Count
params = {
    "radius": 50,
    "types": "restaurant",
    "key": gkey
}

# Use the lat/lng we recovered to identify airports
for index, row in g_lookup_pd_filter.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Long"]
    restaurantname = row["Restaurant Name"]
    restaurantname = restaurantname.title()

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        g_lookup_pd_filter.loc[index, "Type"] = name_address["results"][0]["types"][0].title()
        g_lookup_pd_filter.loc[index, "Rating"] = name_address["results"][0]["rating"]
        g_lookup_pd_filter.loc[index, "Rating Count"] = name_address["results"][0]["user_ratings_total"]
        
        print(f'''[OK!] {restaurantname} found!''')
        
    
    except (KeyError, IndexError):
        print(f'''[ERROR!] {restaurantname} missing field/result.''')

In [ ]:
g_lookup_pd_filter

In [6]:
# Save as CSV complete DoD/Google dataset (OPTION)
#g_lookup_pd_filter.to_csv("./output_data/Google Lat Long Results.csv")

# Save as CSV filtered DoD/Google dataset.  E.g. Restaurant/Bar/Cafe
#g_lookup_pd_filter.to_csv("./output_data/Google Lat Long Results_RestBarCafe.csv")


# Import DoD/Google CSV (OPTION)
dallas_open_data = "./output_data/Google Lat Long Results.csv"
dallas_open_data_df = pd.read_csv(dallas_open_data, dtype="object", encoding="utf-8")

# set output CSV data file filepath variable
output_data_filepath = "output_data/"

## Yelp Data

In [13]:
# import Dallas Open Data CSV data file (with lat/lng added;
# obtained from Google API) into dataframe & visualize data frame

# Option to filter for 'Type' here?
#dallas_open_data_df = g_lookup_pd_filter
#dallas_open_data_df.head()

In [7]:
# create a second data frame to use below and visualize the data frame
# df = dallas_open_data_df
df = dallas_open_data_df
df.head()

,Unnamed: 0,index,Restaurant Name,Inspection Type,Inspection Date,Inspection Score,Street Number,Street Name,Street Unit,Zip,Street Address,Inspection Month,Inspection Year,Lat,Long,Type,Rating,Rating Count
0,0,0,FRESHII,Routine,10/31/2018,96,2414,VICTORY PARK,NaN,75219,2414 VICTORY PARK LN,10/1/18,FY2019,32.7879479,-96.80916429999999,Restaurant,4.1,100
1,1,145,KROGER,Routine,04/27/2017,100,4142,CEDAR SPRINGS,NaN,75219,4142 CEDAR SPRINGS RD,04/1/17,FY2017,32.8133582,-96.81214089999999,NaN,NaN,NaN
2,2,177,ZOE'S KITCHEN,Routine,05/11/2017,98,3878,OAK LAWN,#314,75219,3878 OAK LAWN #314,05/1/17,FY2017,32.8154852,-96.8006898,Restaurant,4.2,334
3,3,191,MANSION MAIN BAR,Routine,10/10/2017,99,2821,TURTLE CREEK,NaN,75219,2821 TURTLE CREEK BLVD,10/1/17,FY2018,32.8041143,-96.8073446,Bar,4.6,433
4,4,213,PERSONALIZED LEARNING PREPARATORY AT SAM HOUSTON,Routine,05/23/2019,96,2827,THROCKMORTON,NaN,75219,2827 THROCKMORTON ST,05/1/19,FY2019,32.810203,-96.81312349999999,NaN,NaN,NaN


In [8]:
#check how many records are coming through
len(df)

50

In [9]:
# set API URL and API key variables to pass through in the API request
api_key_1 = ykey

headers_1 = {'Authorization': 'Bearer %s' % api_key_1}
# headers_2 = {'Authorization': 'Bearer %s' % api_key_2}

url = 'https://api.yelp.com/v3/businesses/search'

In [10]:
# create blank/empty variables for the data input into and returned
# from the API get requests and counters

# # setup of variables (initially as blank); to be updated & used in the for loop with API call for each row
search_name = ""
search_address = ""
search_street_number = ""
search_street_name = ""
search_zip_code = ""
search_lat = ""
search_lng = ""

# setup counters, initially set to 1, to count cities with API calls that successfully retrieve data
attempt_counter = 1
record_counter = 1
set_of_records_counter = 1
record_in_set_counter = 1

# create list to hold all results, in the event we need to reference later
yelp_results_all = []

In [11]:
# loop through rows in df, sending an API request for that restaurant
# using restaurant_name, street_number, street_name and zip_code

# start log for data retrieval from API Call
print("Beginning Data Retrieval")
print("-----------------------------")

# loop through df results & complete API call for each; append df with data when successful
for index, row in df.iterrows():
    # get restaurant data from df, setting respective variables for that restaurant
    search_name = row["Restaurant Name"]
    search_street_number = row["Street Number"]
    search_street_name = row["Street Name"]
    search_zip_code = row["Zip"]
    search_lat = row["Lat"]
    search_lng = row["Long"]
    
    # set API request parameters
    params = {
        "term": f"{search_name}",
        "location": f"{search_street_number} {search_street_name} Dallas, TX {search_zip_code}",
        "radius": 4000,
        "limit": 5,
        "sort_by": "best_match"
    }
    
    # set yelp_response to results from API request
    yelp_response = requests.get(url, params=params, headers=headers_1).json()
    
    # extend yelp_response_all to hold all results in case we need to use this later
    try:
        yelp_results_all.extend(yelp_response["businesses"])
    except:
        pass
    
    # traverse json object, set yelp_results to only business data for the best match result (0 index position)
    try:
        yelp_results = yelp_response["businesses"][0]
#         yelp_results = yelp_response["businesses"]
        print(f"Attempt: {attempt_counter} | Processing record {record_counter} ({record_in_set_counter} of set {set_of_records_counter}) | {search_name}, {search_street_number} {search_street_name} Dallas, TX {search_zip_code}")

        # not all results have all fields; to prevent error, try/except error handling
        try:
            yelp_category_1 = yelp_results["categories"][0]["title"]
            df.loc[index, "yelp_category_1"] = yelp_category_1
        except:
            pass
            
        try:
            yelp_category_2 = yelp_results["categories"][1]["title"]
            df.loc[index, "yelp_category_2"] = yelp_category_2
        except:
            pass
            
        try:
            yelp_category_3 = yelp_results["categories"][2]["title"]
            df.loc[index, "yelp_category_3"] = yelp_category_3
        except:
            pass
        
        try:
            yelp_latitude = yelp_results["coordinates"]["latitude"]
            df.loc[index, "yelp_lat"] = yelp_latitude
        except:
            pass
        
        try:
            yelp_longitude = yelp_results["coordinates"]["longitude"]
            df.loc[index, "yelp_lng"] = yelp_longitude
        except:
            pass
        
        try:
            yelp_is_closed = yelp_results["is_closed"]
            df.loc[index, "yelp_is_closed"] = yelp_is_closed
        except:
            pass
        
        try:
            yelp_address_1 = yelp_results["location"]["address1"]
            df.loc[index, "yelp_address_1"] = yelp_address_1
        except:
            pass
        
        try:
            yelp_address_2 = yelp_results["location"]["address2"]
            df.loc[index, "yelp_address_2"] = yelp_address_2
        except:
            pass
        
        try:
            yelp_zip_code = yelp_results["location"]["zip_code"]
            df.loc[index, "yelp_zip_code"] = yelp_zip_code
        except:
            pass
        
        try:
            yelp_name = yelp_results["name"]
            df.loc[index, "yelp_name"] = yelp_name
        except:
            pass
        
        try:
            yelp_price = yelp_results["price"]
            df.loc[index, "yelp_price"] = yelp_price
        except:
            pass
        
        try:
            yelp_rating = yelp_results["rating"]
            df.loc[index, "yelp_rating"] = yelp_rating
        except:
            pass
        
        try:
            yelp_review_count = yelp_results["review_count"]
            df.loc[index, "yelp_review_count"] = yelp_review_count
        except:
            pass

        # update counters
        attempt_counter += 1
        record_counter += 1
        
        # for set counters, once 50 is reached: reset record_in_set_counter = 1 and add 1 to set_counter
        if record_in_set_counter >= 50:
            set_of_records_counter += 1
            record_in_set_counter = 1
        else:
            record_in_set_counter += 1

    except:
        print(f"Skipping...  Attempt: {attempt_counter} | Record ({search_name} {search_street_number} {search_street_name} Dallas, TX {search_zip_code}) is not found.")
        # update attempt_counter
        attempt_counter += 1
#         pass

# end log for data retrieval; print "Data Retrieval Complete" when at the end of the loop
print("----------------------------")
print("Data Retrieval Complete")
print("----------------------------")

Beginning Data Retrieval
-----------------------------
Attempt: 1 | Processing record 1 (1 of set 1) | FRESHII, 2414 VICTORY PARK Dallas, TX 75219
Attempt: 2 | Processing record 2 (2 of set 1) | KROGER, 4142 CEDAR SPRINGS Dallas, TX 75219
Attempt: 3 | Processing record 3 (3 of set 1) | ZOE'S KITCHEN, 3878 OAK LAWN Dallas, TX 75219
Attempt: 4 | Processing record 4 (4 of set 1) | MANSION MAIN BAR, 2821 TURTLE CREEK Dallas, TX 75219
Skipping...  Attempt: 5 | Record (PERSONALIZED LEARNING PREPARATORY AT SAM HOUSTON 2827 THROCKMORTON Dallas, TX 75219) is not found.
Skipping...  Attempt: 6 | Record (MEDRANO ELEMENTARY SCHOOL 2221 LUCAS Dallas, TX 75219) is not found.
Skipping...  Attempt: 7 | Record (SILVERADO SENIOR LIVING-TURTLE CREEK 3611 DICKASON Dallas, TX 75219) is not found.
Attempt: 8 | Processing record 5 (5 of set 1) | SAL'S PIZZA, 2525 WYCLIFF Dallas, TX 75219
Skipping...  Attempt: 9 | Record (SCOTTISH RITE HOSPITAL SERVERY 2222 WELBURN Dallas, TX 75219) is not found.
Attempt: 10 

In [12]:
# visualize data frame updated with API results
df.head()

,Unnamed: 0,index,Restaurant Name,Inspection Type,Inspection Date,Inspection Score,Street Number,Street Name,Street Unit,Zip,...,yelp_lat,yelp_lng,yelp_is_closed,yelp_address_1,yelp_address_2,yelp_zip_code,yelp_name,yelp_price,yelp_rating,yelp_review_count
0,0,0,FRESHII,Routine,10/31/2018,96,2414,VICTORY PARK,NaN,75219,...,32.787948,-96.809164,False,2414 Victory Park Lane,,75219,Freshii,$$,3.0,63.0
1,1,145,KROGER,Routine,04/27/2017,100,4142,CEDAR SPRINGS,NaN,75219,...,32.813435,-96.812308,False,4142 Cedar Springs Rd,,75219,Kroger,$$,2.5,128.0
2,2,177,ZOE'S KITCHEN,Routine,05/11/2017,98,3878,OAK LAWN,#314,75219,...,32.815348,-96.800765,False,3878 Oak Lawn Ave,Ste 314,75219,Zoes Kitchen,$$,3.0,79.0
3,3,191,MANSION MAIN BAR,Routine,10/10/2017,99,2821,TURTLE CREEK,NaN,75219,...,32.797867,-96.798463,False,2418 Allen St,,75204,Parliament,$$,4.5,508.0
4,4,213,PERSONALIZED LEARNING PREPARATORY AT SAM HOUSTON,Routine,05/23/2019,96,2827,THROCKMORTON,NaN,75219,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# loop through df: compare Dallas Open Data, Google and Yelp info and
# assign a match "confidence" score (FuzzyWuzzy WRatio) & visualize df

# loop through df results & complete Dallas Open Data, Google and Yelp info
for index, row in df.iterrows():
    
    # check to see if there is a Google rating for the record, flag it if so    
    if str(row["Rating"]) != "nan":
        df.loc[index, "google_result_flag"] = 1
    else:
        df.loc[index, "google_result_flag"] = 0
    
    # check to see if there is a yelp result for the row, if so continue and flag it
    if str(row["yelp_name"]) != "nan":
        df.loc[index, "yelp_result_flag"] = 1
        
        # assign FuzzyWuzzy WRatio score for Dallas Open Data value vs Yelp, when yelp result/value is available
        dod_name = row["Restaurant Name"]
        y_name = row["yelp_name"]
#         print(f"NAME: {dod_name} vs {y_name} FuzzyWuzzy WRatio: ", fuzz.WRatio(dod_name.lower(), y_name.lower()))
        df.loc[index, "name_match_score"] = fuzz.WRatio(dod_name.lower(), y_name.lower())
        
        dod_street_number = row["Street Number"]
        dod_street_name = row["Street Name"]
        dod_address = f"{dod_street_number} {dod_street_name}"
        y_address = row["yelp_address_1"]
#         print(f"ADDRESS: {dod_address} vs {y_address} FuzzyWuzzy WRatio: ", fuzz.WRatio(dod_address, y_address))
        df.loc[index, "address_match_score"] = fuzz.WRatio(dod_address, y_address)
        
        dod_lat = str(round(float(row["Lat"]),5))
        dod_lng = str(round(float(row["Long"]),5))
        y_lat = str(round(float(row["yelp_lat"]),5))
        y_lng = str(round(float(row["yelp_lng"]),5))
        
        dod_lat_lng = f"({dod_lat}, {dod_lng})"
        y_lat_lng = f"({y_lat}, {y_lng})"
#         print(f"LAT/LNG: {dod_lat_lng} vs {y_lat_lng} FuzzyWuzzy WRatio: ", fuzz.WRatio(dod_lat_lng, y_lat_lng))
        df.loc[index, "lat_lng_match_score"] = fuzz.WRatio(dod_lat_lng, y_lat_lng)
        
    else:
        df.loc[index, "yelp_result_flag"] = 0

df.head()

,Unnamed: 0,index,Restaurant Name,Inspection Type,Inspection Date,Inspection Score,Street Number,Street Name,Street Unit,Zip,...,yelp_zip_code,yelp_name,yelp_price,yelp_rating,yelp_review_count,google_result_flag,yelp_result_flag,name_match_score,address_match_score,lat_lng_match_score
0,0,0,FRESHII,Routine,10/31/2018,96,2414,VICTORY PARK,NaN,75219,...,75219,Freshii,$$,3.0,63.0,1.0,1.0,100.0,95.0,100.0
1,1,145,KROGER,Routine,04/27/2017,100,4142,CEDAR SPRINGS,NaN,75219,...,75219,Kroger,$$,2.5,128.0,0.0,1.0,100.0,95.0,89.0
2,2,177,ZOE'S KITCHEN,Routine,05/11/2017,98,3878,OAK LAWN,#314,75219,...,75219,Zoes Kitchen,$$,3.0,79.0,1.0,1.0,96.0,95.0,84.0
3,3,191,MANSION MAIN BAR,Routine,10/10/2017,99,2821,TURTLE CREEK,NaN,75219,...,75204,Parliament,$$,4.5,508.0,1.0,1.0,34.0,40.0,63.0
4,4,213,PERSONALIZED LEARNING PREPARATORY AT SAM HOUSTON,Routine,05/23/2019,96,2827,THROCKMORTON,NaN,75219,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN


In [15]:
# loop again through df results & complete Dallas Open Data, Google and Yelp info to flag high_match_score_flag
for index, row in df.iterrows():
    
    # check to see & flag if there is a yelp hit and a high confidence match score >= 70 for name and address
    if (row["yelp_result_flag"] == 1) & (row["name_match_score"] >= 80) & (row["address_match_score"] >= 80):
        df.loc[index, "high_match_score_flag"] = 1
    else:
        df.loc[index, "high_match_score_flag"] = 0
        
df.head()

,Unnamed: 0,index,Restaurant Name,Inspection Type,Inspection Date,Inspection Score,Street Number,Street Name,Street Unit,Zip,...,yelp_name,yelp_price,yelp_rating,yelp_review_count,google_result_flag,yelp_result_flag,name_match_score,address_match_score,lat_lng_match_score,high_match_score_flag
0,0,0,FRESHII,Routine,10/31/2018,96,2414,VICTORY PARK,NaN,75219,...,Freshii,$$,3.0,63.0,1.0,1.0,100.0,95.0,100.0,1.0
1,1,145,KROGER,Routine,04/27/2017,100,4142,CEDAR SPRINGS,NaN,75219,...,Kroger,$$,2.5,128.0,0.0,1.0,100.0,95.0,89.0,1.0
2,2,177,ZOE'S KITCHEN,Routine,05/11/2017,98,3878,OAK LAWN,#314,75219,...,Zoes Kitchen,$$,3.0,79.0,1.0,1.0,96.0,95.0,84.0,1.0
3,3,191,MANSION MAIN BAR,Routine,10/10/2017,99,2821,TURTLE CREEK,NaN,75219,...,Parliament,$$,4.5,508.0,1.0,1.0,34.0,40.0,63.0,0.0
4,4,213,PERSONALIZED LEARNING PREPARATORY AT SAM HOUSTON,Routine,05/23/2019,96,2827,THROCKMORTON,NaN,75219,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0.0


In [16]:
# organize / reorder columns in df & visualize

df = df.loc[:,["Restaurant Name",
               "yelp_name",
               "name_match_score",
               "Street Number",
               "Street Name",
               "Street Unit",
               "Street Address",
               "Zip",
               "yelp_address_1",
               "yelp_address_2",
               "yelp_zip_code",
               "address_match_score",
               "Lat",
               "Long",
               "yelp_lat",
               "yelp_lng",
               "lat_lng_match_score",
               "Inspection Score",
               "Inspection Type",
               "Inspection Date",
               "Inspection Month",
               "Inspection Year",
               "Rating",
               "Rating Count",
               "Type",
               "yelp_rating",
               "yelp_review_count",
               "yelp_price",
               "yelp_category_1",
               "yelp_category_2",
               "yelp_category_3",
               "yelp_is_closed",
               "google_result_flag",
               "yelp_result_flag",
               "high_match_score_flag"
              ]
           ]
df.head()

,Restaurant Name,yelp_name,name_match_score,Street Number,Street Name,Street Unit,Street Address,Zip,yelp_address_1,yelp_address_2,...,yelp_rating,yelp_review_count,yelp_price,yelp_category_1,yelp_category_2,yelp_category_3,yelp_is_closed,google_result_flag,yelp_result_flag,high_match_score_flag
0,FRESHII,Freshii,100.0,2414,VICTORY PARK,NaN,2414 VICTORY PARK LN,75219,2414 Victory Park Lane,,...,3.0,63.0,$$,Salad,Vegetarian,Juice Bars & Smoothies,False,1.0,1.0,1.0
1,KROGER,Kroger,100.0,4142,CEDAR SPRINGS,NaN,4142 CEDAR SPRINGS RD,75219,4142 Cedar Springs Rd,,...,2.5,128.0,$$,Grocery,NaN,NaN,False,0.0,1.0,1.0
2,ZOE'S KITCHEN,Zoes Kitchen,96.0,3878,OAK LAWN,#314,3878 OAK LAWN #314,75219,3878 Oak Lawn Ave,Ste 314,...,3.0,79.0,$$,Mediterranean,Greek,Sandwiches,False,1.0,1.0,1.0
3,MANSION MAIN BAR,Parliament,34.0,2821,TURTLE CREEK,NaN,2821 TURTLE CREEK BLVD,75219,2418 Allen St,,...,4.5,508.0,$$,Cocktail Bars,NaN,NaN,False,1.0,1.0,0.0
4,PERSONALIZED LEARNING PREPARATORY AT SAM HOUSTON,NaN,NaN,2827,THROCKMORTON,NaN,2827 THROCKMORTON ST,75219,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0


In [17]:
# export restaurant_df to output CSV data file (Option)

# df.to_csv(f"{output_data_filepath}yelp_output.csv", encoding="utf-8", index=False)
df.to_csv(f"{output_data_filepath}yelp_output.csv", encoding="UTF-16LE", index=False)

## Analysis